# [dividiti](http://dividiti.com)'s submissions to [MLPerf Inference v0.5](https://github.com/mlperf/inference/tree/master/v0.5)

<a id="overview"></a>
## Overview

This Jupyter notebook covers [dividiti](http://dividiti.com)'s submissions to [MLPerf Inference v0.5](https://github.com/mlperf/inference/tree/master/v0.5). It validates that experimental data obtained via automated, portable and reproducible [Collective Knowledge](http://cknowledge.org) workflows conforms to [General MLPerf Submission Rules](https://github.com/mlperf/policies/blob/master/submission_rules.adoc)
and [MLPerf Inference Rules](https://github.com/mlperf/inference_policies/blob/master/inference_rules.adoc), including runnning the official [`submission_checker.py`](https://github.com/mlperf/inference/blob/master/v0.5/tools/submission/submission-checker.py).

## Table of Contents

1. [Overview](#overview)
1. [Includes](#includes)
1. [Systems](#systems)
  1. [Firefly RK3399](#systems_firefly)
  1. [Linaro HiKey960](#systems_hikey960)
  1. [Huawei Mate 10 Pro](#systems_mate10pro)
  1. [Raspberry Pi 4](#systems_rpi4)

<a id="includes"></a>
## Includes

### Standard

In [ ]:
import os
import sys
import json
import re

from pprint import pprint
from shutil import copy2
from copy import deepcopy

### Scientific

If some of the scientific packages are missing, please install them using:
```
# python3 -m pip install jupyter pandas numpy matplotlib seaborn --user
```

In [ ]:
import IPython as ip
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sb

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('NumPy version: %s' % np.__version__)
print ('Matplotlib version: %s' % mp.__version__)
print ('Seaborn version: %s' % sb.__version__)

In [ ]:
from IPython.display import Image, display
def display_in_full(df):
    pd.options.display.max_columns = len(df.columns)
    pd.options.display.max_rows = len(df.index)
    display(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
default_colormap = cm.autumn
default_fontsize = 16
default_barwidth = 0.8
default_figwidth = 24
default_figheight = 3
default_figdpi = 200
default_figsize = [default_figwidth, default_figheight]

In [ ]:
if mp.__version__[0]=='2': mp.style.use('classic')
mp.rcParams['figure.max_open_warning'] = 200
mp.rcParams['figure.dpi'] = default_figdpi
mp.rcParams['font.size'] = default_fontsize
mp.rcParams['legend.fontsize'] = 'medium'

In [ ]:
save_fig_dir = os.path.join(os.path.expanduser("~"), 'mlperf-dividiti')
if not os.path.exists(save_fig_dir):
    os.mkdir(save_fig_dir)

### Collective Knowledge

If CK is not installed, please install it using:
```
# python -m pip install ck
```

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

<a id="systems"></a>
## Systems

<a id="systems_firefly"></a>
### [Firefly-RK3399](http://en.t-firefly.com/product/rk3399/)

In [ ]:
firefly = {
    "division": "",
    "submitter": "dividiti",
    "status": "available",
    "system_name": "Firefly-RK3399",

    "number_of_nodes": "1",
    "host_processor_model_name": "Arm Cortex-A72 MP2 (big); Arm Cortex-A53 MP4 (LITTLE)",
    "host_processors_per_node": "1",
    "host_processor_core_count": "2 (big); 4 (LITTLE)",
    "host_processor_frequency": "1800 MHz (big), 1400 MHz (LITTLE)",
    "host_processor_caches": "L1I$ 48 KiB, L1D$ 32 KiB, L2$ 1 MiB (big); L1I$ 32 KiB, L1D$ 32 KiB, L2$ 512 KiB (LITTLE)",
    "host_memory_configuration": "-",
    "host_memory_capacity": "4 GiB",
    "host_storage_capacity": "128 GiB",
    "host_storage_type": "SanDisk Extreme microSD",
    "host_processor_interconnect": "-",
    "host_networking": "-",
    "host_networking_topology": "-",

    "accelerators_per_node": "1",
    "accelerator_model_name": "Arm Mali-T860 MP4",
    "accelerator_frequency": "800 MHz",
    "accelerator_host_interconnect": "-",
    "accelerator_interconnect": "-",
    "accelerator_interconnect_topology": "-",
    "accelerator_memory_capacity": "4 GiB (shared with host)",
    "accelerator_memory_configuration": "-",
    "accelerator_on-chip_memories": "-",
    "cooling": "on-board fan",
    "hw_notes": "http://en.t-firefly.com/product/rk3399/; http://opensource.rock-chips.com/wiki_RK3399",

    "framework": "",
    "operating_system": "Ubuntu 16.04.6 LTS",
    "other_software_stack": "GCC 7.4.0; Python 3.5.2",
    "sw_notes": "Powered by Collective Knowledge v1.11.1"
}

#### TFLite v1.15

In [ ]:
closed_firefly_tflite = deepcopy(firefly)
closed_firefly_tflite.update({
    "division" : "closed",
    "framework" : "TFLite v1.15.0-rc2"
})
closed_firefly_tflite

In [ ]:
open_firefly_tflite = deepcopy(firefly)
open_firefly_tflite.update({
    "division" : "open",
    "framework" : "TFLite v1.15.0-rc2"
})
open_firefly_tflite

#### ArmNN v19.08

In [ ]:
closed_firefly_armnn = deepcopy(firefly)
closed_firefly_tflite.update({
    "division" : "closed",
    "framework" : "ArmNN v19.08"
})
closed_firefly_armnn

In [ ]:
open_firefly_armnn = deepcopy(firefly)
open_firefly_tflite.update({
    "division" : "open",
    "framework" : "ArmNN v19.08"
})
open_firefly_armnn

<a id="systems_hikey960"></a>
### [Linaro HiKey960](https://www.96boards.org/product/hikey960/)

In [ ]:
hikey960 = {
    "division": "",
    "submitter": "dividiti",
    "status": "available",
    "system_name": "Linaro HiKey960",

    "number_of_nodes": "1",
    "host_processor_model_name": "Arm Cortex-A73 MP4 (big); Arm Cortex-A53 MP4 (LITTLE)",
    "host_processors_per_node": "1",
    "host_processor_core_count": "4 (big); 4 (LITTLE)",
    "host_processor_frequency": "2362 MHz (big), 1844 MHz (LITTLE)",
    "host_processor_caches": "L1I$ 256=4x64 KiB, L1D$ 256=4x64 KiB, L2$ 2 MiB (big); L1I$ 128=4x32 KiB, L1D$ 128=4x32 KiB, L2$ 1 MiB (LITTLE)",
    "host_memory_configuration": "-",
    "host_memory_capacity": "3 GiB",
    "host_storage_capacity": "128 GiB",
    "host_storage_type": "SanDisk Extreme microSD",
    "host_processor_interconnect": "-",
    "host_networking": "-",
    "host_networking_topology": "-",

    "accelerators_per_node": "1",
    "accelerator_model_name": "Arm Mali-G71 MP8",
    "accelerator_frequency": "800 MHz",
    "accelerator_host_interconnect": "-",
    "accelerator_interconnect": "-",
    "accelerator_interconnect_topology": "-",
    "accelerator_memory_capacity": "3 GiB (shared with host)",
    "accelerator_memory_configuration": "-",
    "accelerator_on-chip_memories": "-",
    "cooling": "small external fan",
    "hw_notes": "http://www.hisilicon.com/en/Products/ProductList/Kirin",

    "framework": "",
    "operating_system": "Debian 9",
    "other_software_stack": "GCC 7.4.0; Python 3.5.3",
    "sw_notes": "Powered by Collective Knowledge v1.11.1"
}

#### TFLite v1.15

In [ ]:
closed_hikey960_tflite = deepcopy(hikey960)
closed_hikey960_tflite.update({
    "division" : "closed",
    "framework" : "TFLite v1.15.0-rc2"
})
closed_hikey960_tflite

In [ ]:
open_hikey960_tflite = deepcopy(hikey960)
open_hikey960_tflite.update({
    "division" : "open",
    "framework" : "TFLite v1.15.0-rc2"
})
open_hikey960_tflite

#### ArmNN v19.08

In [ ]:
closed_hikey960_armnn = deepcopy(hikey960)
closed_hikey960_tflite.update({
    "division" : "closed",
    "framework" : "ArmNN v19.08"
})
closed_hikey960_armnn

In [ ]:
open_hikey960_armnn = deepcopy(hikey960)
open_hikey960_tflite.update({
    "division" : "open",
    "framework" : "ArmNN v19.08"
})
open_hikey960_armnn

<a id="systems_mate10pro"></a>
### Huawei Mate 10 Pro

In [ ]:
mate10pro = {
    "division": "",
    "submitter": "dividiti",
    "status": "available",
    "system_name": "Huawei Mate 10 Pro",

    "number_of_nodes": "1",
    "host_processor_model_name": "Arm Cortex-A73 MP4 (big); Arm Cortex-A53 MP4 (LITTLE)",
    "host_processors_per_node": "1",
    "host_processor_core_count": "4 (big); 4 (LITTLE)",
    "host_processor_frequency": "2360 MHz (big), 1800 MHz (LITTLE)",
    "host_processor_caches": "L1I$ 256=4x64 KiB, L1D$ 256=4x64 KiB, L2$ 2 MiB (big); L1I$ 128=4x32 KiB, L1D$ 128=4x32 KiB, L2$ 1 MiB (LITTLE)",
    "host_memory_configuration": "-",
    "host_memory_capacity": "6 GiB",
    "host_storage_capacity": "128 GiB",
    "host_storage_type": "-",
    "host_processor_interconnect": "-",
    "host_networking": "-",
    "host_networking_topology": "-",

    "accelerators_per_node": "1",
    "accelerator_model_name": "Arm Mali-G72 MP12",
    "accelerator_frequency": "850 MHz",
    "accelerator_host_interconnect": "-",
    "accelerator_interconnect": "-",
    "accelerator_interconnect_topology": "-",
    "accelerator_memory_capacity": "6 GiB (shared with host)",
    "accelerator_memory_configuration": "-",
    "accelerator_on-chip_memories": "-",
    "cooling": "phone case",
    "hw_notes": "https://en.wikichip.org/wiki/hisilicon/kirin/970",

    "framework": "",
    "operating_system": "Android 8",
    "other_software_stack": "Android NDK 17c (LLVM 6.0.2)",
    "sw_notes": "Powered by Collective Knowledge v1.11.1"
}

#### TFLite v1.13

In [ ]:
closed_mate10pro_tflite = deepcopy(mate10pro)
closed_mate10pro_tflite.update({
    "division" : "closed",
    "framework" : "TFLite v1.13.1"
})
closed_mate10pro_tflite

In [ ]:
open_mate10pro_tflite = deepcopy(mate10pro)
open_mate10pro_tflite.update({
    "division" : "open",
    "framework" : "TFLite v1.13.1"
})
open_mate10pro_tflite

#### ArmNN v19.08

In [ ]:
closed_mate10pro_armnn = deepcopy(mate10pro)
closed_mate10pro_armnn.update({
    "division" : "closed",
    "framework" : "ArmNN v19.08"
})
closed_mate10pro_armnn

In [ ]:
open_mate10pro_armnn = deepcopy(mate10pro)
open_mate10pro_armnn.update({
    "division" : "open",
    "framework" : "ArmNN v19.08"
})
open_mate10pro_armnn

#### Raspberry Pi 4

In [ ]:
rpi4 = {
    "division": "",
    "submitter": "dividiti",
    "status": "available",
    "system_name": "Raspberry Pi 4",

    "number_of_nodes": "1",
    "host_processor_model_name": "Arm Cortex-A72 MP4",
    "host_processors_per_node": "1",
    "host_processor_core_count": "4",
    "host_processor_frequency": "1500 MHz",
    "host_processor_caches": "L1I$ 128=4x32 KiB, L1D$ 128=4x32 KiB, L2$ 1 MiB",
    "host_memory_configuration": "-",
    "host_memory_capacity": "4 GiB",
    "host_storage_capacity": "128 GiB",
    "host_storage_type": "SanDisk Extreme Pro microSD",
    "host_processor_interconnect": "-",
    "host_networking": "-",
    "host_networking_topology": "-",

    "accelerators_per_node": "0",
    "accelerator_model_name": "-",
    "accelerator_frequency": "-",
    "accelerator_host_interconnect": "-",
    "accelerator_interconnect": "-",
    "accelerator_interconnect_topology": "-",
    "accelerator_memory_capacity": "-",
    "accelerator_memory_configuration": "-",
    "accelerator_on-chip_memories": "-",
    "cooling": "http://www.raspberrypiwiki.com/index.php/Armor_Case_B",
    "hw_notes": "https://www.raspberrypi.org/products/raspberry-pi-4-model-b/specifications/",

    "framework": "",
    "operating_system": "Debian 10",
    "other_software_stack": "GCC 8.3.0; Python 3.7.3",
    "sw_notes": "Powered by Collective Knowledge v1.11.1"
}

#### TFLite v1.15

In [ ]:
closed_rpi4_tflite = deepcopy(rpi4)
closed_rpi4_tflite.update({
    "division"  : "closed",
    "framework" : "TFLite v1.15.0-rc2"
})
closed_rpi4_tflite

In [ ]:
open_rpi4_tflite = deepcopy(rpi4)
open_rpi4_tflite.update({
    "division"  : "open",
    "framework" : "TFLite v1.15.0-rc2"
})
open_rpi4_tflite

#### ArmNN v19.08

In [ ]:
closed_rpi4_armnn = deepcopy(rpi4)
closed_rpi4_armnn.update({
    "division"  : "closed",
    "framework" : "ArmNN v19.08"
})
closed_rpi4_armnn

In [ ]:
open_rpi4_armnn = deepcopy(rpi4)
open_rpi4_armnn.update({
    "division"  : "open",
    "framework" : "ArmNN v19.08"
})
open_rpi4_armnn

### All

In [ ]:
division_systems = {
    # Closed.
    'closed-firefly-tflite-v1.15'   : closed_firefly_tflite,
    'closed-firefly-armnn-v19.08'   : closed_firefly_armnn,
    'closed-hikey960-tflite-v1.15'  : closed_hikey960_tflite,
    'closed-hikey960-armnn-v19.08'  : closed_hikey960_armnn,
    'closed-mate10pro-tflite-v1.13' : closed_mate10pro_tflite,
    'closed-mate10pro-armnn-v19.08' : closed_mate10pro_armnn,
    'closed-rpi4-tflite-v1.15'      : closed_rpi4_tflite,
    'closed-rpi4-armnn-v19.08'      : closed_rpi4_armnn,    
    # Open.
    'open-firefly-tflite-v1.15'     : open_firefly_tflite,
    'open-firefly-armnn-v19.08'     : open_firefly_armnn,
    'open-hikey960-tflite-v1.15'    : open_hikey960_tflite,
    'open-hikey960-armnn-v19.08'    : open_hikey960_armnn,
    'open-mate10pro-tflite-v1.13'   : open_mate10pro_tflite,
    'open-mate10pro-armnn-v19.08'   : open_mate10pro_armnn,
    'open-rpi4-tflite-v1.15'        : open_rpi4_tflite,
    'open-rpi4-armnn-v19.08'        : open_rpi4_armnn,
}

### Default `system_desc_id.json` (to catch uninitialized descriptions)

In [ ]:
default_system_json = {
    "division": "reqired",
    "submitter": "required",
    "status": "required",
    "system_name": "required",

    "number_of_nodes": "required",
    "host_processor_model_name": "required",
    "host_processors_per_node": "required",
    "host_processor_core_count": "required",
    "host_processor_frequency": "",
    "host_processor_caches": "",
    "host_memory_configuration": "",
    "host_memory_capacity": "required",
    "host_storage_capacity": "required",
    "host_storage_type": "required",
    "host_processor_interconnect": "",
    "host_networking": "",
    "host_networking_topology": "",

    "accelerators_per_node": "required",
    "accelerator_model_name": "required",
    "accelerator_frequency": "",
    "accelerator_host_interconnect": "",
    "accelerator_interconnect": "",
    "accelerator_interconnect_topology": "",
    "accelerator_memory_capacity": "required",
    "accelerator_memory_configuration": "",
    "accelerator_on-chip_memories": "",
    "cooling": "",
    "hw_notes": "",

    "framework": "required",
    "operating_system": "required",
    "other_software_stack": "required",
    "sw_notes": ""
}

<a id="implementations"></a>
## Implementations

In [ ]:
# Generate implementation_benchmarks dictionary.
implementation_benchmarks = {}

for implementation in [ 'image-classification-tflite', 'image-classification-armnn-tflite' ]:
    # Add MobileNet.
    implementation_mobilenet = implementation+'-'+'mobilenet'
    implementation_benchmarks[implementation_mobilenet] = {
        "input_data_types": "fp32",
        "weight_data_types": "fp32",
        "retraining": "no",
        "starting_weights_filename": "https://zenodo.org/record/2269307/files/mobilenet_v1_1.0_224.tgz",
        "weight_transformations": "TFLite"
    }
    # Add ResNet.
    implementation_resnet = implementation+'-'+'resnet'
    implementation_benchmarks[implementation_resnet] = {
        "input_data_types": "fp32",
        "weight_data_types": "fp32",
        "retraining": "no",
        "starting_weights_filename": "https://zenodo.org/record/2535873/files/resnet50_v1.pb",
        "weight_transformations": "TF -> TFLite"
    }
    # Add any MobileNets-v1,v2 model.
    def add_implementation_mobilenet(implementation_benchmarks, version, multiplier, resolution):
        base_url = 'https://zenodo.org/record/2269307/files' if version == 1 else 'https://zenodo.org/record/2266646/files'
        url = '{}/mobilenet_v{}_{}_{}.tgz'.format(base_url, version, multiplier, resolution)
        benchmark = 'mobilenet-v{}-{}-{}'.format(version, multiplier, resolution)
        if implementation == 'image-classification-tflite':
            weights_transformations = 'TFLite'
        elif implementation == 'image-classification-armnn-tflite':
            weights_transformations = 'TFLite -> ArmNN'
        else:
            raise "Unknown implementation '%s'!" % implementation
        implementation_benchmark = implementation+'-'+benchmark
        implementation_benchmarks[implementation_benchmark] = {
            "input_data_types": "fp32",
            "weight_data_types": "fp32",
            "retraining": "no",
            "starting_weights_filename": url,
            "weight_transformations": weights_transformations
        }
        return
    # MobileNet-v1.
    version = 1
    for multiplier in [ 1.0, 0.75, 0.5, 0.25 ]:
        for resolution in [ 224, 192, 160, 128 ]:
            add_implementation_mobilenet(implementation_benchmarks, version, multiplier, resolution)
    # MobileNet-v2.
    version = 2
    for multiplier in [ 1.0, 0.75, 0.5, 0.35 ]:
        for resolution in [ 224, 192, 160, 128, 96 ]:
            add_implementation_mobilenet(implementation_benchmarks, version, multiplier, resolution)
    add_implementation_mobilenet(implementation_benchmarks, version=2, multiplier=1.3, resolution=224)
    add_implementation_mobilenet(implementation_benchmarks, version=2, multiplier=1.4, resolution=224)

# from pprint import pprint
# pprint(implementation_benchmarks)

In [ ]:
implementation_readmes = {}
implementation_readmes['image-classification-tflite'] = \
"""# MLPerf Inference - Image Clasification - TFLite
- [Source code](https://github.com/ctuning/ck-mlperf/tree/master/program/image-classification-tflite-loadgen).
- [Instructions](https://github.com/mlperf/inference/blob/master/v0.5/classification_and_detection/optional_harness_ck/classification/tflite/README.md).
"""

implementation_readmes['image-classification-armnn-tflite'] = \
"""# MLPerf Inference - Image Clasification - ArmNN-TFLite
- [Source code](https://github.com/ctuning/ck-mlperf/tree/master/program/image-classification-armnn-tflite-loadgen).
- [Instructions](https://github.com/ARM-software/armnn-mlperf/blob/master/README.md).
"""

In [ ]:
implementation_paths = {}
for implementation in [ 'image-classification-tflite', 'image-classification-armnn-tflite' ]:
    implementation_loadgen = implementation+'-loadgen'
    r = ck.access({'action':'find', 'repo_uoa':'ck-mlperf', 'module_uoa':'program', 'data_uoa':implementation_loadgen})
    if r['return']>0:
        print('Error: %s' % r['error'])
        exit(1)
    implementation_paths[implementation] = r['path']

In [ ]:
# Snapshot of https://github.com/dividiti/inference/blob/61220457dec221ed1984c62bd9d382698bd71bc6/v0.5/mlperf.conf
mlperf_conf_6122045 = '''
# The format of this config file is 'key = value'.
# The key has the format 'model.scenario.key'. Value is mostly int64_t.
# Model maybe '*' as wildcard. In that case the value applies to all models.
# All times are in milli seconds

*.SingleStream.target_latency = 10
*.SingleStream.target_latency_percentile = 90
*.SingleStream.min_duration = 60000
*.SingleStream.min_query_count = 1024

*.MultiStream.target_qps = 20
*.MultiStream.target_latency_percentile = 99
*.MultiStream.samples_per_query = 4
*.MultiStream.max_async_queries = 1
*.MultiStream.target_latency = 50
*.MultiStream.min_duration = 60000
*.MultiStream.min_query_count = 270336
ssd-resnet34.MultiStream.target_qps = 15
ssd-resnet34.MultiStream.target_latency = 66
gnmt.MultiStream.min_query_count = 90112
gnmt.MultiStream.target_latency = 100
gnmt.MultiStream.target_qps = 10
gnmt.MultiStream.target_latency_percentile = 97

*.Server.target_qps = 1.0
*.Server.target_latency = 10
*.Server.target_latency_percentile = 99
*.Server.target_duration = 0
*.Server.min_duration = 60000
*.Server.min_query_count = 270336
resnet50.Server.target_latency = 15
ssd-resnet34.Server.target_latency = 100
gnmt.Server.min_query_count = 90112
gnmt.Server.target_latency = 250
gnmt.Server.target_latency_percentile = 97

*.Offline.target_qps = 1.0
*.Offline.target_latency_percentile = 90
*.Offline.min_duration = 60000
*.Offline.min_query_count = 1
'''

### Default `system_desc_id_imp.json` (to catch uninitialized descriptions)

In [ ]:
default_implementation_benchmark_json = {
    "input_data_types": "required",
    "retraining": "required",
    "starting_weights_filename": "required",
    "weight_data_types": "required",
    "weight_transformations": "required"
}

<a id="get"></a>
## Get the experimental data

Download experimental data and add CK repositories as follows.

### Image Classification - Closed (MobileNet, ResNet)

#### `firefly`

```
$ wget https://www.dropbox.com/s/3md826fk7k1taf3/mlperf.closed.image-classification.firefly.tflite-v1.15.zip
$ ck add repo --zip=mlperf.closed.image-classification.firefly.tflite-v1.15.zip

$ https://www.dropbox.com/s/jusoz329mhixpxm/mlperf.closed.image-classification.firefly.armnn-v19.08.neon.zip
$ ck add repo --zip=mlperf.closed.image-classification.firefly.armnn-v19.08.neon.zip

$ wget https://www.dropbox.com/s/08lzbz7jl2w5jhu/mlperf.closed.image-classification.firefly.armnn-v19.08.opencl.zip
$ ck add repo --zip=mlperf.closed.image-classification.firefly.armnn-v19.08.opencl.zip
```

#### `hikey960`

```
$ wget https://www.dropbox.com/s/lqnffl6wbaeceul/mlperf.closed.image-classification.hikey960.tflite-v1.15.zip
$ ck add repo --zip=mlperf.closed.image-classification.hikey960.tflite-v1.15.zip

$ wget https://www.dropbox.com/s/6m6uv1d33yc82f8/mlperf.closed.image-classification.hikey960.armnn-v19.08.neon.zip
$ ck add repo --zip=mlperf.closed.image-classification.hikey960.armnn-v19.08.neon.zip

$ wget https://www.dropbox.com/s/bz56y4damfqggr8/mlperf.closed.image-classification.hikey960.armnn-v19.08.opencl.zip
$ ck add repo --zip=mlperf.closed.image-classification.hikey960.armnn-v19.08.opencl.zip
```

#### `rpi4`

**TODO**: missing performance experiments with TFLite v1.15; no experiments with ArmNN v19.08 with Neon.

#### `mate10pro` (**BAD LOADGEN**)

```
$ wget https://www.dropbox.com/s/nts8e7unb7vm68f/mlperf.closed.image-classification.mate10pro.tflite-v1.13.mobilenet.zip
$ ck add repo --zip=mlperf.closed.image-classification.mate10pro.tflite-v1.13.mobilenet.zip

$ wget https://www.dropbox.com/s/t2o2elqdyitqlpi/mlperf.closed.image-classification.mate10pro.armnn-v19.08.neon.zip
$ ck add repo --zip=mlperf.closed.image-classification.mate10pro.armnn-v19.08.neon.zip
```

**TODO:** accuracy experiment with TFLite v1.15 for ResNet aborted (estimated 17 hours); no OpenCL experiments with ArmNN.

### Image Classification - Open (MobileNets-v1,v2)

#### `firefly`

```
$ wget https://www.dropbox.com/s/q8ieqgnr3zn6w4y/mlperf.open.image-classification.firefly.tflite-v1.15.zip
$ ck add repo --zip=mlperf.open.image-classification.firefly.tflite-v1.15.zip
```

**TODO:** experiments with ArmNN  w/ OpenCL are in progress.

#### `hikey960`

```
$ wget https://www.dropbox.com/s/2gbbpsd2pjurvc8/mlperf.open.image-classification.hikey960.tflite-v1.15.zip
$ ck add repo --zip=mlperf.open.image-classification.hikey960.tflite-v1.15.zip
```

**TODO:** experiments with ArmNN  w/ OpenCL are in progress.

#### `rpi4`

**TODO:** experiments with TFLite are in progress.

<a id="check"></a>
## Check the experimental data

In [ ]:
repos = [
    #
    # Image Classification - Closed (MobileNet, ResNet)
    #
    # firefly
    'mlperf.closed.image-classification.firefly.tflite-v1.15',
    'mlperf.closed.image-classification.firefly.armnn-v19.08.neon',
    'mlperf.closed.image-classification.firefly.armnn-v19.08.opencl',
    # hikey960
    'mlperf.closed.image-classification.hikey960.tflite-v1.15',
    'mlperf.closed.image-classification.hikey960.armnn-v19.08.neon',
    'mlperf.closed.image-classification.hikey960.armnn-v19.08.opencl',
    # mate10pro
    'mlperf.closed.image-classification.mate10pro.armnn-v19.08.neon', # BAD LOADGEN
    'mlperf.closed.image-classification.mate10pro.tflite-v1.13.mobilenet', # BAD LOADGEN
    # Image Classification - Open (MobileNets-v1,v2)
    # firefly
    'mlperf.open.image-classification.firefly.tflite-v1.15',
    # hikey960
    'mlperf.open.image-classification.hikey960.tflite-v1.15',
]

In [ ]:
!ck recache repo
for repo_uoa in repos:
    print("=" * 100)
    print(repo_uoa)
    print("=" * 100)
    !ck list $repo_uoa:experiment:* | sort
    print("-" * 100)
    print("")

In [ ]:
# Locate upstream master.
r = ck.access({'action':'locate', 'module_uoa':'env', 'tags':'mlperf,inference,source,upstream.master'})
if r['return']>0:
    print('Error: %s' % r['error'])
    exit(1)
# Pick any source location and look under 'inference/v0.5/mlperf.conf'.
upstream_path = os.path.join(list(r['install_locations'].values())[0], 'inference')
upstream_path

In [ ]:
def get_experimental_results(repo_uoa, module_uoa='experiment', tags='mlperf', submitter='dividiti', path=None):
    if not path:
        path_list = !ck find repo:$repo_uoa
        path = path_list[0]
    root_dir = os.path.join(path, 'submissions_inference_0_5')
    if not os.path.exists(root_dir): os.mkdir(root_dir)
    print("Storing results under '%s'" % root_dir)
    
    r = ck.access({'action':'search', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'tags':tags})
    if r['return']>0:
        print('Error: %s' % r['error'])
        exit(1)
    experiments = r['lst']

    dfs = []
    for experiment in experiments:
        data_uoa = experiment['data_uoa']
        r = ck.access({'action':'list_points', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'data_uoa':data_uoa})
        if r['return']>0:
            print('Error: %s' % r['error'])
            exit(1)
        print("*" * 100)

        tags = r['dict']['tags']
        #print(tags)
        if 'accuracy' in tags:
            if 'neon' in tags or 'opencl' in tags:
                # Expected format: [ "mlperf", "open", "image-classification", "firefly", "armnn-v19.08", "neon", "mobilenet-v1-0.5-128", "singlestream", "accuracy", "using-opencv" ]
                (_, division, task, platform, library, backend, benchmark, scenario, mode, preprocessing) = tags
            else:
                # Expected format: [ "mlperf", "open", "image-classification", "firefly", "tflite-v1.15", "mobilenet-v1-0.5-128", "singlestream", "accuracy", "using-opencv" ]
                (_, division, task, platform, library, benchmark, scenario, mode, preprocessing) = tags
        elif 'performance' in tags:            
            if 'neon' in tags or 'opencl' in tags:
                # Expected format: [ "mlperf", "open", "image-classification", "firefly", "armnn-v19.08", "neon", "mobilenet-v1-0.5-128", "singlestream", "performance" ]
                (_, division, task, platform, library, backend, benchmark, scenario, mode) = tags
            else:
                # Expected format: [ "mlperf", "open", "image-classification", "firefly", "tflite-v1.15", "mobilenet-v1-0.5-128", "singlestream", "performance" ]
                (_, division, task, platform, library, benchmark, scenario, mode) = tags
        else:
            raise "Expected 'accuracy' or 'performance' in tags!"
            
        organization = submitter
        system = platform+'-'+library
        division_system = division+'-'+system
        if library.startswith('tflite'):
            implementation = task+'-tflite'
        elif library.startswith('armnn'):
            implementation = task+'-armnn-tflite'
        else: # Official app with CK adaptations.
            implementation = 'mlperf-inference-vision'
        implementation_benchmark = implementation+'-'+benchmark
        
        #
        # Directory structure according to the Inference section of the General MLPerf Submission Rules:
        # https://github.com/mlperf/policies/blob/master/submission_rules.adoc#552-inference
        #
        # <division>/
        #   <organization>/
        #
        division_dir = os.path.join(root_dir, division)
        if not os.path.exists(division_dir): os.mkdir(division_dir)
        organization_dir = os.path.join(division_dir, organization)
        if not os.path.exists(organization_dir): os.mkdir(organization_dir)
        
        #
        #     "systems"/
        #       <system_desc_id>.json
        #
        systems_dir = os.path.join(organization_dir, 'systems')
        if not os.path.exists(systems_dir): os.mkdir(systems_dir)
        system_json_name = '%s.json' % system
        system_json_path = os.path.join(systems_dir, system_json_name)
        with open(system_json_path, 'w') as system_json_file:
#             pprint(division_system)
#             pprint(division_systems)
            system_json = division_systems.get(division_system, default_system_json)
            json.dump(system_json, system_json_file, indent=2)
            print('%s' % systems_dir)
            if system_json == default_system_json:
                print('  |_ %s [DEFAULT]' % system_json_name)
                raise
            else:
                print('  |_ %s [%s]' % (system_json_name, division_system))
        
        #
        #     "code"/
        #       <benchmark_name_per_reference>/
        #         <implementation_id>/
        #           <Code interface with loadgen and other arbitrary stuff>
        #
        code_dir = os.path.join(organization_dir, 'code')
        if not os.path.exists(code_dir): os.mkdir(code_dir)
        # FIXME: For now, not always "per reference".
        benchmark_dir = os.path.join(code_dir, benchmark)
        if not os.path.exists(benchmark_dir): os.mkdir(benchmark_dir)
        implementation_dir = os.path.join(benchmark_dir, implementation)
        if not os.path.exists(implementation_dir): os.mkdir(implementation_dir)
        print('%s' % code_dir)
        # Create 'README.md'.
        implementation_readme_name = 'README.md'
        implementation_readme_path = os.path.join(implementation_dir, implementation_readme_name)
        implementation_readme = implementation_readmes.get(implementation, '')
#         pprint(implementation)
#         pprint(implementation_readmes)
        with open(implementation_readme_path, 'w') as implementation_readme_file:
            implementation_readme_file.writelines(implementation_readme)
        if implementation_readme == '':
            print('  |_ %s [EMPTY]' % implementation_readme_name)
            raise
        else:
            print('  |_ %s' % implementation_readme_name)
        # TODO: Copy run.sh

        #
        #     "measurements"/
        #       <system_desc_id>/
        #         <benchmark>/
        #           <scenario>/
        #             <system_desc_id>_<implementation_id>.json
        #             README.md
        #             user.conf
        #             mlperf.conf
        #             calibration_process.adoc (?)
        #
        measurements_dir = os.path.join(organization_dir, 'measurements')
        if not os.path.exists(measurements_dir): os.mkdir(measurements_dir)
        system_dir = os.path.join(measurements_dir, system)
        if not os.path.exists(system_dir): os.mkdir(system_dir)
        benchmark_dir = os.path.join(system_dir, benchmark)
        if not os.path.exists(benchmark_dir): os.mkdir(benchmark_dir)
        scenario_dir = os.path.join(benchmark_dir, scenario)
        if not os.path.exists(scenario_dir): os.mkdir(scenario_dir)
        print(scenario_dir)
        # Create '<system_desc_id>_<implementation_id>.json'.
        system_implementation_json_name = system+'_'+implementation+'.json'
        system_implementation_json_path = os.path.join(scenario_dir, system_implementation_json_name)
        with open(system_implementation_json_path, 'w') as system_implementation_json_file:
#             pprint(implementation_benchmark)
#             pprint(implementation_benchmarks)
            implementation_benchmark_json = implementation_benchmarks.get(implementation_benchmark, default_implementation_benchmark_json)
            json.dump(implementation_benchmark_json, system_implementation_json_file, indent=2)
            if implementation_benchmark_json == default_implementation_benchmark_json:
                print('  |_ %s [DEFAULT]' % system_implementation_json_name)
                raise
            else:
                print('  |_ %s [%s]' % (system_implementation_json_name, implementation_benchmark))
        # Create empty 'README.md'.
        system_implementation_readme_name = 'README.md'
        system_implementation_readme_path = os.path.join(scenario_dir, system_implementation_readme_name)
        with open(system_implementation_readme_path, 'a'):
            os.utime(system_implementation_readme_path, None)
            print('  |_ %s [EMPTY]' % system_implementation_readme_name)
        
        # Copy 'user.conf' from the implementation source.
        user_conf_name = 'user.conf'
        implementation_path = implementation_paths.get(implementation, '')
        if implementation_path == '':
            raise "Invalid implementation path!"
        else:
            user_conf_path = os.path.join(implementation_path, user_conf_name)
            copy2(user_conf_path, scenario_dir)
            print('  |_ %s [from %s]' % (user_conf_name, user_conf_path))
        
        # Copy 'mlperf.conf' from MLPerf Inference source.
        mlperf_conf_name = 'mlperf.conf'
        mlperf_conf_path = os.path.join(scenario_dir, mlperf_conf_name)
        if implementation in [ 'image-classification-tflite', 'image-classification-armnn-tflite' ]:
            # Write a snapshot from https://github.com/dividiti/inference/blob/61220457dec221ed1984c62bd9d382698bd71bc6/v0.5/mlperf.conf
            with open(mlperf_conf_path, 'w') as mlperf_conf_file:
                mlperf_conf_file.writelines(mlperf_conf_6122045)
            print('  |_ %s [from %s]' % (mlperf_conf_name, 'github.com/mlperf/inference@6122045'))
        else:
            upstream_mlperf_conf_path = os.path.join(upstream_path, 'v0.5', 'mlperf.conf')
            copy2(upstream_mlperf_conf_path, mlperf_conf_path)
            print('  |_ %s [from %s]' % (mlperf_conf_name, upstream_mlperf_conf_path))
        
        #
        #     "results"/
        #       <system_desc_id>/
        #         <benchmark>/
        #           <scenario>/
        #             performance/
        #               run_x/ # 1 run for single stream and offline, 5 otherwise
        #                 mlperf_log_summary.txt
        #                 mlperf_log_detail.txt
        #                 mlperf_log_trace.json
        #             accuracy/
        #               mlperf_log_accuracy.json
        #       compliance_checker_log.txt (?)
        #
        results_dir = os.path.join(organization_dir, 'results')
        if not os.path.exists(results_dir): os.mkdir(results_dir)
        system_dir = os.path.join(results_dir, system)
        if not os.path.exists(system_dir): os.mkdir(system_dir)
        benchmark_dir = os.path.join(system_dir, benchmark)
        if not os.path.exists(benchmark_dir): os.mkdir(benchmark_dir)
        scenario_dir = os.path.join(benchmark_dir, scenario)
        if not os.path.exists(scenario_dir): os.mkdir(scenario_dir)
        mode_dir = os.path.join(scenario_dir, mode)
        if not os.path.exists(mode_dir): os.mkdir(mode_dir)
        # For each point (should be one point for each performance run).
        points = r['points']
        for (point, point_idx) in zip(points, range(1,len(points)+1)):
            point_file_path = os.path.join(r['path'], 'ckp-%s.0001.json' % point)
            with open(point_file_path) as point_file:
                point_data_raw = json.load(point_file)
            characteristics_list = point_data_raw['characteristics_list']
            characteristics = characteristics_list[0]
            # Set the leaf directory.
            if mode == 'performance':
                run_dir = os.path.join(mode_dir, 'run_%d' % point_idx)
                if not os.path.exists(run_dir): os.mkdir(run_dir)
                last_dir = run_dir
            else:
                last_dir = mode_dir
            print(last_dir)
            # Dump files in the leaf directory.
            mlperf_log = characteristics['run'].get('mlperf_log',{})
            # Summary file (with errors and warnings in accuracy mode, with statistics in performance mode).
            summary_txt_name = 'mlperf_log_summary.txt'
            summary_txt_path = os.path.join(last_dir, summary_txt_name)
            with open(summary_txt_path, 'w') as summary_txt_file:
                summary_txt_file.writelines(mlperf_log.get('summary',''))
                print('  |_ %s' % summary_txt_name)
            # Detail file (with settings).
            detail_txt_name = 'mlperf_log_detail.txt'
            detail_txt_path = os.path.join(last_dir, detail_txt_name)
            with open(detail_txt_path, 'w') as detail_txt_file:
                detail_txt_file.writelines(mlperf_log.get('detail',''))
                print('  |_ %s' % detail_txt_name)
            # Accuracy file (with accuracy dictionary).
            # FIXME: Move the next 5 lines into the (if mode == 'accuracy') block,
            # once the submission checker no longer complains as follows:
            # "performance/run_1 has file list mismatch (['mlperf_log_accuracy.json'])"
            accuracy_json_name = 'mlperf_log_accuracy.json'
            accuracy_json_path = os.path.join(last_dir, accuracy_json_name)
            with open(accuracy_json_path, 'w') as accuracy_json_file:
                json.dump(mlperf_log.get('accuracy',{}), accuracy_json_file, indent=2)
                print('  |_ %s' % accuracy_json_name)
            if mode == 'accuracy':
                accuracy_imagenet_py = os.path.join(upstream_path, 'v0.5', 'classification_and_detection', 'tools', 'accuracy-imagenet.py')
                imagenet_val_file = '$HOME/CK_TOOLS/dataset-imagenet-ilsvrc2012-aux/val.txt' # FIXME: Do not hardcode - locate via CK.
                accuracy_txt_name = 'accuracy.txt'
                accuracy_txt_path = os.path.join(last_dir, accuracy_txt_name)
                accuracy_txt = !python3 $accuracy_imagenet_py --imagenet-val-file $imagenet_val_file --mlperf-accuracy-file $accuracy_json_path
                with open(accuracy_txt_path, 'w') as accuracy_txt_file:
                    accuracy_txt_file.writelines(accuracy_txt)
                    # Print the first line containing accuracy info.
                    print('  |_ %s ["%s"]' % (accuracy_txt_name, accuracy_txt[0]))
#             # Trace file (should omit trace from v0.5).
#             trace_json_name = 'mlperf_log_trace.json'
#             trace_json_path = os.path.join(last_dir, trace_json_name)
#             with open(trace_json_path, 'w') as trace_json_file:
#                 json.dump(mlperf_log.get('trace',{}), trace_json_file, indent=2)

    print("*" * 100)
    submission_checker_py = os.path.join(upstream_path, 'v0.5', 'tools', 'submission', 'submission-checker.py')
    checker_log = !python3 $submission_checker_py --input $root_dir --submitter $submitter
    checker_log = "\n".join(checker_log)
    print(checker_log)
    checker_log_name = 'compliance_checker_log.txt'
    checker_log_path = os.path.join(organization_dir, checker_log_name)
    with open(checker_log_path, 'w') as checker_log_file:
        checker_log_file.writelines(checker_log)
        print(organization_dir)
        print('  |_%s' % checker_log_name)
    
    return dfs

repo_uoa = 'mlperf.open.image-classification.firefly.tflite-v1.15'
dfs = get_experimental_results(repo_uoa)